# Accessing Data

In [2]:
#This gets us into the right directory from home, in order to run the import python script from Mat

#sys = system (module), gives information and control over python interpreter/terminal itself
import sys
sys.path.append("/home/565/pv3484/aus_substation_electricity")

#% is a magic command, special shortcut command that lets you control/interact with notebook environment (ie. gives control of terminal without writing full python code)
%cd aus_substation_electricity/

!pwd

/home/565/pv3484/aus_substation_electricity
/home/565/pv3484/aus_substation_electricity


In [3]:
#This section imports the substations that Mat put together

%run /home/565/pv3484/aus_substation_electricity/import_substation.py

processing nsw substations for ['ausgrid'] from None to None
ausgrid
following columns in demand are not in info index:
['MT_HU', 'SI_NO']
removing these columns from demand
number of substations in ausgrid substation info: 134
number of substations in ausgrid substation data: 132
following sites match selection criteria:
               energy_asset          Name  Area  Dwellings  Persons  Residential  Commercial  Industrial  Primary Production  Education  \
ID                                                                                                                                        
BLAKE         AG_BLAKEHURST    Blakehurst     7      10081    28521        0.850       0.005       0.021               0.000      0.022   
PUNCH          AG_PUNCHBOWL     Punchbowl     9      17514    50395        0.826       0.048       0.038               0.000      0.025   
MEADO         AG_MEADOWBANK    Meadowbank    15      22420    56948        0.825       0.023       0.015               0

# Anomaly Demand Timing

## Demand Anomalies
- soft coded to produce demand anomaly plot for all substations around a public holiday
- 30 +/- window
- includes full name of substation and residential fraction
- only creates anomalies of a single date (eg. Aus day 2008)

In [21]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

def plot_anomalies_by_holiday(demand, info, ref_date, holiday_name,
                              base_folder="/home/aus_substation_electricity/figures/Demand anomalies",
                              window_days=30):
    """
    Generate and save demand anomaly plots for all substations around a holiday.

    Parameters
    ----------
    demand : pd.DataFrame
        Time-indexed demand data (datetime index, substations as columns).
    info : pd.DataFrame
        Metadata table with substation info (must include 'Name', 'Residential', 'Population' if available).
    ref_date : str or pd.Timestamp
        Reference date for the holiday (e.g., '2008-01-26').
    holiday_name : str
        Name of the holiday (used for folder organization).
    base_folder : str
        Base folder where plots will be saved.
    window_days : int
        Number of days before/after the reference date to include in the window.
    """

    # --- Step 1: Ensure datetime index ---
    demand.index = pd.to_datetime(demand.index)

    # --- Step 2: Resample to hourly demand ---
    hourly = demand.resample("h").mean()

    # --- Step 3: Reference date ---
    ref_date = pd.Timestamp(ref_date)

    # --- Step 4: Define ±window around reference date ---
    start = ref_date - pd.Timedelta(days=window_days)
    end   = ref_date + pd.Timedelta(days=window_days)

    # --- Step 5: Create holiday-specific folder ---
    holiday_folder = os.path.join(base_folder, holiday_name)
    os.makedirs(holiday_folder, exist_ok=True)

    # --- Step 6: Loop through each substation ---
    for substation in hourly.columns:
        # Slice window
        window = hourly.loc[start:end, [substation]].copy()

        # Baseline (mean across window)
        baseline_mean = window[substation].mean()
        window["anomaly"] = window[substation] - baseline_mean

        # Metadata
        full_name = info.loc[substation, "Name"]
        fraction = info.loc[substation, "Residential"]

        # Plot
        plt.figure(figsize=(14,5))
        plt.plot(window.index, window["anomaly"], color="red", linewidth=1.5,
                 label=f"{full_name} Anomaly")
        plt.axhline(0, color="black", linewidth=1)
        plt.axvline(ref_date, color="blue", linestyle="--", alpha=0.7,
                    label=f"Reference Date {ref_date.date()}")
        plt.grid(axis='y', linestyle='-', linewidth=0.5, color='gray', alpha=0.3)

        ax = plt.gca()
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=3))
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%b-%d"))
        plt.xticks(rotation=45)

        plt.title(f"Demand Anomaly (±{window_days} days around {ref_date.date()})\n"
                  f"{full_name} (Residential: {fraction:.0%})", fontsize=14)
        plt.xlabel("Date")
        plt.ylabel("Electricity Demand Anomaly")
        plt.legend(title="Reference", bbox_to_anchor=(1.02, 1), loc="upper left", fontsize="small")
        plt.tight_layout()
        plt.show()
        
        # Save
        #filename = f"{substation}_{ref_date.date()}_anomaly.png"
        #filepath = os.path.join(holiday_folder, filename)
        #plt.savefig(filepath, dpi=300)
        #plt.close()

## Comparing demand anomalies on Australia day against januarary weekends
- not the same 30 day +/- period
- can only change substation and date intervals

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def compare_australia_day_vs_jan_weekends(demand, info, station, years):
    """
    Compare demand anomalies on Australia Day vs. January weekends for a given year interval.
    """
    demand.index = pd.to_datetime(demand.index)
    hourly = demand[[station]].resample("h").mean()

    # Collect ±30-day windows around Australia Day for each year
    windows = []
    for year in years:
        ref_date = pd.Timestamp(f"{year}-01-26")
        start = ref_date - pd.Timedelta(days=30)
        end   = ref_date + pd.Timedelta(days=30)
        window = hourly.loc[start:end].copy()
        window["mdh"] = window.index.strftime("%m-%d %H:%M")
        window = window.set_index("mdh")
        windows.append(window)

    combined = pd.concat(windows, axis=1)
    baseline_mean = combined.mean(axis=1).mean()
    anomalies = combined.subtract(baseline_mean, axis=0)
    avg_anomaly = anomalies.mean(axis=1)

    # Australia Day anomalies (24 hours)
    ad_hours = []
    for year in years:
        ref_date = pd.Timestamp(f"{year}-01-26")
        day_str = ref_date.strftime("%m-%d")
        daily = avg_anomaly.loc[avg_anomaly.index.str.startswith(day_str)]
        daily.index = range(24)
        ad_hours.append(daily)
    australia_day = pd.concat(ad_hours, axis=1).mean(axis=1)

    # January weekend anomalies (24 hours)
    weekend_hours = []
    for year in years:
        jan = hourly.loc[f"{year}-01-01":f"{year}-01-31"].copy()
        jan_anomaly = jan.subtract(baseline_mean)
        for day in pd.date_range(f"{year}-01-01", f"{year}-01-31"):
            if day.weekday() >= 5:  # Sat=5, Sun=6
                start = day.replace(hour=0, minute=0)
                end   = day.replace(hour=23, minute=59)
                daily = jan_anomaly.loc[start:end, station]
                if not daily.empty:
                    daily.index = daily.index.hour
                    weekend_hours.append(daily)
    weekends = pd.concat(weekend_hours, axis=1).mean(axis=1)

    # Plot
    plt.figure(figsize=(12,5))
    plt.plot(australia_day.index, australia_day.values, color="red", linewidth=2, marker="o",
             label="Australia Day Anomaly")
    plt.plot(weekends.index, weekends.values, color="blue", linewidth=2, marker="s",
             label="January Weekends Anomaly")

    plt.axhline(0, color="black", linewidth=1)
    plt.grid(axis='y', linestyle='-', linewidth=0.5, color='gray', alpha=0.3)
    plt.xticks(np.arange(0, 24, 1), [f"{h:02d}:00" for h in range(24)], rotation=45)

    full_name = info.loc[station, "Name"]
    plt.title(f"{full_name} Demand Anomaly: Australia Day vs. January Weekends ({years[0]}–{years[-1]})", fontsize=14)
    plt.xlabel("Hour of Day")
    plt.ylabel("Electricity Demand Anomaly")
    plt.legend()
    plt.tight_layout()
    plt.close()
    


# --- Loop through consecutive intervals ---
station = "BLAKE"
year_range = range(2004, 2019)  # 2004–2018 inclusive

for y1, y2 in zip(year_range[:-1], year_range[1:]):
    compare_australia_day_vs_jan_weekends(demand, info, station, [y1, y2])

## Australia Day Anomalies across 2 year periods
- not soft coded, has a focus date and year interval

### 2005-2006

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# --- Step 1: Ensure datetime index ---
demand.index = pd.to_datetime(demand.index)

# --- Step 2: Choose focus date and years ---
focus_date = "01-26"   # e.g. Australia Day, change to "12-25" for Christmas
years = [2005, 2006]   # list of years to average

# --- Step 3: Resample to hourly demand ---
hourly = demand.resample("h").mean()

# --- Step 4: Loop through each substation ---
for station in hourly.columns:
    # Collect ±30-day windows for each year
    windows = []
    for year in years:
        ref_date = pd.Timestamp(f"{year}-{focus_date}")
        start = ref_date - pd.Timedelta(days=30)
        end   = ref_date + pd.Timedelta(days=30)
        window = hourly.loc[start:end, [station]].copy()
        window["mdh"] = window.index.strftime("%m-%d %H:%M")  # align by calendar date
        window = window.set_index("mdh")
        windows.append(window)

    # Combine all windows
    combined = pd.concat(windows, axis=1)

    # Compute baseline (mean demand across all years' windows)
    baseline_mean = combined.mean(axis=1).mean()

    # Compute anomalies for each year relative to baseline
    anomalies = combined.subtract(baseline_mean, axis=0)

    # Average anomalies across all chosen years
    avg_anomaly = anomalies.mean(axis=1)

    # Build datetime index for plotting (use first year for calendar axis)
    ref_date = pd.Timestamp(f"{years[0]}-{focus_date}")
    plot_index = pd.date_range(ref_date - pd.Timedelta(days=30),
                               ref_date + pd.Timedelta(days=30),
                               freq="h")

    # Get full substation name from metadata
    full_name = info.loc[station, "Name"]

    # --- Plot ---
    plt.figure(figsize=(14,5))
    plt.plot(plot_index, avg_anomaly.values, color="red", linewidth=1.5,
             label=f"Average Anomaly ({years[0]}–{years[-1]})")

    # Baseline at 0
    plt.axhline(0, color="black", linewidth=1)

    # Mark focus date
    plt.axvline(ref_date, color="blue", linestyle="--", alpha=0.7,
                label=f"Focus Date {focus_date}")

    # Grid and ticks
    plt.grid(axis='y', linestyle='-', linewidth=0.5, color='gray', alpha=0.3)
    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.DayLocator(interval=3))   # tick every 3 days
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b-%d"))
    plt.xticks(rotation=45)

    # Title with full substation name
    plt.title(f"{full_name} Average Demand Anomaly (±30 days around {focus_date}, {years[0]}–{years[-1]})",
              fontsize=14)
    plt.xlabel("Date")
    plt.ylabel("Electricity Demand Anomaly")
    plt.legend()
    plt.tight_layout()
    plt.close()

### 2006-2007

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# --- Step 1: Ensure datetime index ---
demand.index = pd.to_datetime(demand.index)

# --- Step 2: Choose focus date and years ---
focus_date = "01-26"   # e.g. Australia Day, change to "12-25" for Christmas
years = [2015, 2016]   # list of years to average

# --- Step 3: Resample to hourly demand ---
hourly = demand.resample("h").mean()

# --- Step 4: Loop through each substation ---
for station in hourly.columns:
    # Collect ±30-day windows for each year
    windows = []
    for year in years:
        ref_date = pd.Timestamp(f"{year}-{focus_date}")
        start = ref_date - pd.Timedelta(days=30)
        end   = ref_date + pd.Timedelta(days=30)
        window = hourly.loc[start:end, [station]].copy()
        window["mdh"] = window.index.strftime("%m-%d %H:%M")  # align by calendar date
        window = window.set_index("mdh")
        windows.append(window)

    # Combine all windows
    combined = pd.concat(windows, axis=1)

    # Compute baseline (mean demand across all years' windows)
    baseline_mean = combined.mean(axis=1).mean()

    # Compute anomalies for each year relative to baseline
    anomalies = combined.subtract(baseline_mean, axis=0)

    # Average anomalies across all chosen years
    avg_anomaly = anomalies.mean(axis=1)

    # Build datetime index for plotting (use first year for calendar axis)
    ref_date = pd.Timestamp(f"{years[0]}-{focus_date}")
    plot_index = pd.date_range(ref_date - pd.Timedelta(days=30),
                               ref_date + pd.Timedelta(days=30),
                               freq="h")

    # Get full substation name from metadata
    full_name = info.loc[station, "Name"]

    # --- Plot ---
    plt.figure(figsize=(14,5))
    plt.plot(plot_index, avg_anomaly.values, color="red", linewidth=1.5,
             label=f"Average Anomaly ({years[0]}–{years[-1]})")

    # Baseline at 0
    plt.axhline(0, color="black", linewidth=1)

    # Mark focus date
    plt.axvline(ref_date, color="blue", linestyle="--", alpha=0.7,
                label=f"Focus Date {focus_date}")

    # Grid and ticks
    plt.grid(axis='y', linestyle='-', linewidth=0.5, color='gray', alpha=0.3)
    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.DayLocator(interval=3))   # tick every 3 days
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b-%d"))
    plt.xticks(rotation=45)

    # Title with full substation name
    plt.title(f"{full_name} Average Demand Anomaly (±30 days around {focus_date}, {years[0]}–{years[-1]})",
              fontsize=14)
    plt.xlabel("Date")
    plt.ylabel("Electricity Demand Anomaly")
    plt.legend()
    plt.tight_layout()
    plt.close()

### 2012 - 2013

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# --- Step 1: Ensure datetime index ---
demand.index = pd.to_datetime(demand.index)

# --- Step 2: Choose focus date and years ---
focus_date = "01-26"   # e.g. Australia Day, change to "12-25" for Christmas
years = [2015, 2016]   # list of years to average

# --- Step 3: Resample to hourly demand ---
hourly = demand.resample("h").mean()

# --- Step 4: Loop through each substation ---
for station in hourly.columns:
    # Collect ±30-day windows for each year
    windows = []
    for year in years:
        ref_date = pd.Timestamp(f"{year}-{focus_date}")
        start = ref_date - pd.Timedelta(days=30)
        end   = ref_date + pd.Timedelta(days=30)
        window = hourly.loc[start:end, [station]].copy()
        window["mdh"] = window.index.strftime("%m-%d %H:%M")  # align by calendar date
        window = window.set_index("mdh")
        windows.append(window)

    # Combine all windows
    combined = pd.concat(windows, axis=1)

    # Compute baseline (mean demand across all years' windows)
    baseline_mean = combined.mean(axis=1).mean()

    # Compute anomalies for each year relative to baseline
    anomalies = combined.subtract(baseline_mean, axis=0)

    # Average anomalies across all chosen years
    avg_anomaly = anomalies.mean(axis=1)

    # Build datetime index for plotting (use first year for calendar axis)
    ref_date = pd.Timestamp(f"{years[0]}-{focus_date}")
    plot_index = pd.date_range(ref_date - pd.Timedelta(days=30),
                               ref_date + pd.Timedelta(days=30),
                               freq="h")

    # Get full substation name from metadata
    full_name = info.loc[station, "Name"]

    # --- Plot ---
    plt.figure(figsize=(14,5))
    plt.plot(plot_index, avg_anomaly.values, color="red", linewidth=1.5,
             label=f"Average Anomaly ({years[0]}–{years[-1]})")

    # Baseline at 0
    plt.axhline(0, color="black", linewidth=1)

    # Mark focus date
    plt.axvline(ref_date, color="blue", linestyle="--", alpha=0.7,
                label=f"Focus Date {focus_date}")

    # Grid and ticks
    plt.grid(axis='y', linestyle='-', linewidth=0.5, color='gray', alpha=0.3)
    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.DayLocator(interval=3))   # tick every 3 days
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b-%d"))
    plt.xticks(rotation=45)

    # Title with full substation name
    plt.title(f"{full_name} Average Demand Anomaly (±30 days around {focus_date}, {years[0]}–{years[-1]})",
              fontsize=14)
    plt.xlabel("Date")
    plt.ylabel("Electricity Demand Anomaly")
    plt.legend()
    plt.tight_layout()
    plt.close()

# Comparing weekend, weekday and pub holiday anomalies
- soft coding the station, year intervals, public holiday (will ensure 30 days +/- will be around the chosen public holiday)
- comparing the weekend, week-day and public holiday demand anomalies within the same 30 day +/- window period
- accounts for moving holidays (eg. good friday)

In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from dateutil.easter import easter
import os

def compare_holiday_weekends_weekdays_window(
    demand, info, station, years, holiday_func, holiday_name,
    output_dir="/home/565/pv3484/aus_substation_electricity/figures/anomaly_demand"
):
    """
    Compare demand anomalies on a holiday vs. weekends and weekdays
    within the same ±30-day window around the holiday.
    Saves plots as PNG files into holiday-specific folders.

    Parameters
    ----------
    demand : pd.DataFrame
        Time-indexed demand data (datetime index, substations as columns).
    info : pd.DataFrame
        Metadata table with substation info (must include 'Name').
    station : str
        Substation ID (must match a column in demand and index in info).
    years : list of int
        Years to compare (e.g., [2005, 2006]).
    holiday_func : callable
        Function that returns the holiday date for a given year (e.g., easter).
    holiday_name : str
        Name of the holiday (used for folder organization).
    output_dir : str
        Base folder where plots will be saved.
    """

    # Ensure datetime index
    demand.index = pd.to_datetime(demand.index)
    hourly = demand[[station]].resample("h").mean()

    # Collect ±30-day windows around the holiday
    windows = []
    for year in years:
        ref_date = holiday_func(year)
        start = ref_date - pd.Timedelta(days=30)
        end   = ref_date + pd.Timedelta(days=30)
        window = hourly.loc[start:end].copy()
        window["mdh"] = window.index.strftime("%m-%d %H:%M")
        window = window.set_index("mdh")
        windows.append(window)

    combined = pd.concat(windows, axis=1)
    baseline_mean = combined.mean(axis=1).mean()
    anomalies = combined.subtract(baseline_mean, axis=0)
    avg_anomaly = anomalies.mean(axis=1)

    # Holiday anomalies (24 hours)
    holiday_hours = []
    for year in years:
        ref_date = holiday_func(year)
        day_str = ref_date.strftime("%m-%d")
        daily = avg_anomaly.loc[avg_anomaly.index.str.startswith(day_str)]
        if not daily.empty:
            # Ensure exactly 24 hours, fill missing with NaN
            daily = daily.reindex(range(24))
            holiday_hours.append(daily)
    if holiday_hours:
        holiday_profile = pd.concat(holiday_hours, axis=1).mean(axis=1)
    else:
        holiday_profile = pd.Series(np.zeros(24), index=range(24))

    # Weekend and weekday anomalies within the same ±30-day window
    weekend_hours = []
    weekday_hours = []
    for year in years:
        ref_date = holiday_func(year)
        start = ref_date - pd.Timedelta(days=30)
        end   = ref_date + pd.Timedelta(days=30)
        window = hourly.loc[start:end].copy()
        window_anomaly = window.subtract(baseline_mean)
        for day in pd.date_range(start, end):
            start_day = day.replace(hour=0, minute=0)
            end_day   = day.replace(hour=23, minute=59)
            daily = window_anomaly.loc[start_day:end_day, station]
            if not daily.empty:
                daily.index = daily.index.hour
                if day.weekday() >= 5:  # Sat=5, Sun=6
                    weekend_hours.append(daily)
                else:
                    weekday_hours.append(daily)
    weekend_profile = pd.concat(weekend_hours, axis=1).mean(axis=1) if weekend_hours else pd.Series(np.zeros(24), index=range(24))
    weekday_profile = pd.concat(weekday_hours, axis=1).mean(axis=1) if weekday_hours else pd.Series(np.zeros(24), index=range(24))

    # --- Plot ---
    plt.figure(figsize=(12,5))
    plt.plot(holiday_profile.index, holiday_profile.values, color="red", linewidth=2, marker="o",
             label=f"{holiday_name} (±30-Day Window)")
    if not weekend_profile.empty:
        plt.plot(weekend_profile.index, weekend_profile.values, color="blue", linewidth=2, marker="s",
                 label="Weekends (±30-Day Window)")
    if not weekday_profile.empty:
        plt.plot(weekday_profile.index, weekday_profile.values, color="green", linewidth=2, marker="^",
                 label="Weekdays (±30-Day Window)")

    plt.axhline(0, color="black", linewidth=1)
    plt.grid(axis='y', linestyle='-', linewidth=0.5, color='gray', alpha=0.3)
    plt.xticks(np.arange(0, 24, 1), [f"{h:02d}:00" for h in range(24)], rotation=45)

    full_name = info.loc[station, "Name"]
    plt.title(
        f"{full_name} Demand Anomaly: {holiday_name} vs. Weekends & Weekdays "
        f"(±30-Day Window, {years[0]}–{years[-1]})",
        fontsize=14
    )
    plt.xlabel("Hour of Day")
    plt.ylabel("Electricity Demand Anomaly")
    plt.legend()
    plt.tight_layout()

    # --- Save instead of show ---
    # Holiday-specific folder (no year interval in folder name)
    holiday_folder = os.path.join(output_dir, holiday_name)
    os.makedirs(holiday_folder, exist_ok=True)

    filename = f"{station}_{years[0]}-{years[-1]}.png"
    filepath = os.path.join(holiday_folder, filename)

    plt.savefig(filepath, dpi=300)
    plt.close()

    return filepath

## Using the function above

In [32]:
# Define the station
station = "BLAKE"

# Define the year intervals you want to loop through
intervals = [
    [2004, 2005],
    [2007, 2008],
    [2014, 2015],
    [2017, 2018], #Christmas and Boxing Day do not have data for 2017-2018
]

# Define the all national public holidays (including moving i.e. easter, is accounted for)
holidays = {
    "New Year's Day": lambda y: pd.Timestamp(f"{y}-01-01"),
    "Australia Day": lambda y: pd.Timestamp(f"{y}-01-26"),
    "Good Friday": lambda y: easter(y) - pd.Timedelta(days=2),
    "Easter Saturday": lambda y: easter(y) - pd.Timedelta(days=1),
    "Easter Sunday": lambda y: easter(y),
    "Easter Monday": lambda y: easter(y) + pd.Timedelta(days=1),
    "ANZAC Day": lambda y: pd.Timestamp(f"{y}-04-25"),
    "Christmas Day": lambda y: pd.Timestamp(f"{y}-12-25"),
    "Boxing Day": lambda y: pd.Timestamp(f"{y}-12-26"),
}

# Loop through intervals and holidays
for years in intervals:
    for holiday_name, holiday_func in holidays.items():
        compare_holiday_weekends_weekdays_window(
            demand, info, station, years,
            holiday_func=holiday_func,
            holiday_name=holiday_name
        )